## Homework Assignment [45 points]: 
### Problem Statement 1 [15 points]:

In class, we have studied Depth First Search (DFS). If we apply it to the following problem, we will find the solution:

<img src="graph.png" width=800px>

```python
vertices = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J']
edges = [
    ('A', 'B'), ('A', 'C'), ('A', 'D'),
    ('B', 'G'), ('C', 'F'), ('D', 'E'), 
    ('E', 'F'), ('F', 'G'), ('F', 'I'), 
    ('G', 'H'), ('H', 'I'), ('I', 'J'),
    ('E', 'J')]
#Get the GraphProblem, and graph_search, node and StackFrontier from: https://github.com/omarnust/AI/blob/main/02-search_graph.ipynb
problem = GraphProblem(vertices, edges, 'A', 'H')
solution, explored = graph_search(problem)
print(solution[1])
['A', 'D', 'E', 'J', 'I', 'H', 'G']
```
The solution is not optimal.

In this homework, you explore two additional search algorithms. `Depth Limited Search (DLS)` and `Iterative Deepening Search (IDS) Algorithm`. Both of these algorithms are based on DFS. Please review the document titled `depth limited and iterative deepening.pdf` and implement `DLS` and `IDS`

Note: You will have to update the class `Node` to keep track of the depth of the node.

Implement `dls_search` an `ids_search` to find the solution

```python
# This is how the grader will run your code:
# [10 pts] Depth Limited Search (DLS)
problem = GraphProblem(vertices, edges, 'A', 'H')
solution, explored = dls_search(problem, 1)
print(solution[1])

# [5 pts] Iterative Deepening Search (IDS) Algorithm
problem = GraphProblem(vertices, edges, 'A', 'H')
solution, explored = ids_search(problem)
print(solution[1])
```


In [72]:
class GraphProblem():
    
    def __init__(self, vertices, edges, start, goal, heuristic_values=[]):
        self.vertices = vertices
        self.edges = edges
        self.start = start
        self.goal = goal
        self.heuristic_values = heuristic_values
            
    def initial_state(self):
      return self.start

    def goal_test(self, state):
      return state == self.goal
      
    def result(self, state):    
        connected_actions = []
        for edge in self.edges:
            if edge[0] == state:
                connected_actions.append((edge[1], edge[1]))  # (action, resulting state)
            elif edge[1] == state:
                connected_actions.append((edge[0], edge[0]))  # (action, resulting state)
        connected_actions.sort()
        return connected_actions
        
    def path_cost(self, state, newstate):
        for edge in self.edges:
            if (edge[0] == state and edge[1] == newstate) or (edge[1] == state and edge[0] == newstate):
                return edge[2]  # Return the cost associated with the edge
        return float('inf')  # Return infinity if no edge exists between the states

    def heuristic(self, state):
        return self.heuristic_values[self.vertices.index(state)]
    
    

In [73]:
class Node():
    def __init__(self, state, parent, action, gcost=0, hcost=0, depth=0):
        self.state = state
        self.parent = parent # parent of the current node
        self.action = action # action that was taken to reach this node
        self.gcost = gcost # cost
        self.hcost = hcost # heuristic
        self.depth = depth
    def __lt__(self, other):
        return self.gcost+self.hcost < other.gcost+other.hcost

In [91]:
def recursive_dls(node, problem, limit, explored):
        if problem.goal_test(node.state):
            actions = []
            cells = []
            while node.parent is not None:
                actions.append(node.action)
                cells.append(node.state)
                node = node.parent
            actions.append(node.action)
            cells.append(node.state)
            actions.reverse()
            cells.reverse()
            return (actions, cells), explored
        
        elif node.depth == limit:
            return None
        explored.add(node.state)
        
        for action, state in problem.result(node.state):
            if state not in explored:
                child = Node(state=state, parent=node, action=action, gcost=0, hcost=0, depth=node.depth+1)
                result = recursive_dls(child, problem, limit, explored)
                if result is not None:
                    return result
        return None

In [92]:
def dls_search(problem, limit):
    # Initialize frontier to just the starting position
    start = Node(state=problem.initial_state(), parent=None, action=None, depth=0)
    explored = set()
    return recursive_dls(start, problem, limit, explored) or (None, explored)

In [104]:
vertices = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J']
edges = [
    ('A', 'B'), ('A', 'C'), ('A', 'D'),
    ('B', 'G'), ('C', 'F'), ('D', 'E'), 
    ('E', 'F'), ('F', 'G'), ('F', 'I'), 
    ('G', 'H'), ('H', 'I'), ('I', 'J'),
    ('E', 'J')]
problem = GraphProblem(vertices, edges, 'A', 'H')
solution, explored = dls_search(problem, 4)
print(solution, explored)

([None, 'B', 'G', 'H'], ['A', 'B', 'G', 'H']) {'F', 'A', 'G', 'B'}


In [101]:
#Solution 1b
def ids_search(problem):
    depth = 0
    while True:
        # Perform DLS with increasing depth limits
        solution, explored = dls_search(problem, depth)
        if solution is not None:
            return solution, explored
        depth += 1

In [172]:
problem = GraphProblem(vertices, edges, 'A', 'H')
solution, explored = ids_search(problem)
print(solution, explored)  # Print the path (solution)

([None], ['A']) set()



### Problem Statement 2 [20 pts]:
You are given a binary matrix grid of size m x n. In this grid, an island is defined as a group of 1's (representing land) that are connected horizontally or vertically. You can assume that the grid is surrounded by water on all four edges.

The area of an island is determined by the number of cells with the value 1 that make up the island.

Define a class `IslandProblem` that encapsulates the problem. This class should include methods `initial_state`, `set_initial_state` and `result`.

Now write a function `search_island` that will return the area of the island.

```python
# This is how the grader will call you methods
grid = [
    [1, 1, 0, 0, 0],
    [1, 1, 0, 1, 1],
    [0, 0, 0, 1, 1],
    [0, 1, 1, 1, 0],
    [0, 1, 0, 0, 0]]

problem = IslandProblem(grid)
problem.set_initial_state((0,0))
area = search_island(problem) # should output 4
problem = IslandProblem(grid)
problem.set_initial_state((2,3))
area = search_island(problem) # should output 8
```
You should use the class `Node`, `StackFrontier/QueueFrontier` defined in the next cell. 
You implementation should work with any input (grid, start_state etc)

In [57]:
class Node():
    def __init__(self, state, parent, action):
        self.state = state
        self.parent = parent # parent of the current node
        self.action = action # action that was taken to reach this node

# Last in first out frontier
class StackFrontier():
    def __init__(self):
        self.frontier = []

    def add(self, node):
        self.frontier.append(node)

    def contains_state(self, state):
        return any(node.state == state for node in self.frontier)

    def empty(self):
        return len(self.frontier) == 0

    def remove(self):
        if self.empty():
            raise Exception("empty frontier")
        else:
            node = self.frontier.pop()
            return node
# First in first out frontier
class QueueFrontier(StackFrontier):

    def remove(self):
        if self.empty():
            raise Exception("empty frontier")
        else:
            node = self.frontier.pop(0)
            return node

In [152]:
# Your solution
import copy

class IslandProblem:
    def __init__(self, grid):
        self.height = len(grid)
        self.width = max(len(row) for row in grid)
        self.grid = copy.deepcopy(grid)
        
    def initial_state(self):
        return self.start
    
    def set_initial_state(self, state):
        self.start = state
    
    def result(self, state):
        row, col = state
        candidates = [
            ("up", (row - 1, col)),
            ("down", (row + 1, col)),
            ("left", (row, col - 1)),
            ("right", (row, col + 1))
        ]
        result = []
        for action, (r, c) in candidates:
            if 0 <= r < self.height and 0 <= c < self.width and self.grid[r][c] >=0 :
                result.append((action, (r, c)))
        return result

def search_island(problem: IslandProblem):
    start = Node(state=problem.initial_state(), parent=None, action=None)
    if problem.grid[start.state[0]][start.state[1]] == 0:
        return 0
    if (start.state[0] >= problem.height or start.state[1] >= problem.width):
        print("Initial state out of bounds")
        return 0
    frontier = QueueFrontier()
    frontier.add(start)
    
    area = 1
    problem.grid[start.state[0]][start.state[1]] = 0 
    
    explored = set()
    while not frontier.empty():
        node = frontier.remove()
        # Add neighbors to frontier
        for action, state in problem.result(node.state):
            x, y = state
            if not frontier.contains_state(state) and state not in explored and problem.grid[x][y] != 0:
                child = Node(state=state, parent=node, action=action)
                frontier.add(child)
                problem.grid[x][y] = 0
                area = area + 1
    return area

In [168]:
grid = [
    [1, 1, 0, 0, 0],
    [1, 1, 0, 1, 1],
    [0, 0, 0, 1, 1],
    [0, 1, 1, 1, 0],
    [0, 1, 0, 0, 0]]

problem = IslandProblem(grid)
problem.set_initial_state((0,0))
area = search_island(problem) # should output 4
print(area)
problem = IslandProblem(grid)
problem.set_initial_state((2,3))
area = search_island(problem) # should output 8
print(area)

4
8


### Problem Statement 3 [10 pts]:
Now write a function `search_max_island` that will use `IslandProblem` and `search_island` that you have define above to return the maximum area of the island in the grid

```python
# This is how the grader will run your code
grid = [[0,0,1,0,0,0,0,1,0,0,0,0,0],
        [0,0,0,0,0,0,0,1,1,1,0,0,0],
        [0,1,1,0,1,0,0,0,0,0,0,0,0],
        [0,1,0,0,1,1,0,0,1,0,1,0,0],
        [0,1,0,0,1,1,0,0,1,1,1,0,0],
        [0,0,0,0,0,0,0,0,0,0,1,0,0],
        [0,0,0,0,0,0,0,1,1,1,0,0,0],
        [0,0,0,0,0,0,0,1,1,0,0,0,0]]

problem = IslandProblem(grid)
max_area = search_max_island(problem) # should output 6
```

In [169]:
# Your solution
def search_max_island(problem: IslandProblem):
    max_area = 0
    for row in range(problem.height):
        for col in range(problem.width):
            if problem.grid[row][col] == 1:
                problem.set_initial_state((row,col))
                area = search_island(problem)
                if (area > max_area):
                    max_area = area
    return max_area
            

In [170]:
grid = [[0,0,1,0,0,0,0,1,0,0,0,0,0],
        [0,0,0,0,0,0,0,1,1,1,0,0,0],
        [0,1,1,0,1,0,0,0,0,0,0,0,0],
        [0,1,0,0,1,1,0,0,1,0,1,0,0],
        [0,1,0,0,1,1,0,0,1,1,1,0,0],
        [0,0,0,0,0,0,0,0,0,0,1,0,0],
        [0,0,0,0,0,0,0,1,1,1,0,0,0],
        [0,0,0,0,0,0,0,1,1,0,0,0,0]]

problem = IslandProblem(grid)
max_area = search_max_island(problem) # should output 6
print(max_area)

6
